### LAS VEGAS I15 DS cleaning script (2018)

In [ ]:
# Installing libraries: 
# pip install geopy 

In [ ]:
import pandas as pd

def delane_ds(df):
    # Group the data by the two columns and keep only the first occurrence of each group/ Delane 
    # 5/6= lat/lon columns
    reduced_df = df.drop_duplicates(subset=[5, 6], keep='first')

    # Convert column 1 values to strings and remove anything after the first rigt _ and and store the rest
    reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]

    # Add . after the first two numbers in the lat lon column
    reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
    reduced_df.iloc[:, 6] = reduced_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

    return reduced_df


In [ ]:
import pandas as pd

def get_sensor_group(data, sensor_loc, ignore=None):
    # Filter the DataFrame based on column 3 and ignore "ignore" string
    mask = data[3].str.startswith(sensor_loc) & (~data[3].str.contains(ignore) if ignore is not None else True)
    matching_group = data[mask]
    return matching_group

In [ ]:
# Read Data here
import pandas as pd

# PD warning supression
pd.options.mode.chained_assignment = None

######
detector_data = pd.read_csv('../detector/detectors2018.csv', header=None)
detector_data=delane_ds(detector_data)
#####
#sensor_loc = "CC-215 WB"
#sensor_loc="CC-215 EB"
#sensor_loc="I-15 NB"
#sensor_loc= "I-515 SB"
#sensor_loc="US-95 SB"
sensor_loc="US-95 NB"
ignore = 'Ramp'

# Get list of sensors using the sensor group
sensor_group = get_sensor_group(detector_data, sensor_loc, ignore)
    

In [22]:
# SENSOR DATA CLEANING

import pandas as pd 
import os 
import matplotlib.pyplot as plt
import numpy as np
import miceforest as mf

# CONFIGURATION:Change the following directories everytime as necessary

uncleaned_sensor_groups_list=["E:/I-15 Datataset/2018/I-15 NB without Ramp.csv",
                              "E:/I-15 Datataset/2018/I-15 SB without Ramp.csv",
                              "E:/I-15 Datataset/2018/I-515 NB without Ramp.csv",
                              "E:/I-15 Datataset/2018/I-515 SB without Ramp.csv",
                              "E:/I-15 Datataset/2018/US-95 NB without Ramp.csv",
                              "E:/I-15 Datataset/2018/US-95 SB without Ramp.csv",
                              "E:/I-15 Datataset/2018/CC-215 EB without Ramp.csv",
                              "E:/I-15 Datataset/2018/CC-215 WB without Ramp.csv"]

#uncleaned_sensor_date_time=pd.DataFrame(uncleaned_sensor_groups_list)
uncleaned_sensor_date_time=pd.read_csv("E:/I-15 Datataset/2018/dates.csv")

# Directory to save the cleaned ds
clean_sensor_data_dir= "E:/I-15 Datataset/2018/cleaned/"

########################################################################################

# Copying the uncleaned dataset to a new pd so that both copies remains
sensor_date_time=uncleaned_sensor_date_time.copy(deep=True)

# Task 1:time and date valued of the Index 18685 is missing, by manual check of the dataset and placed on the first row of the ds
# First row is to be removed due to the erronous entry at row 0
# There is a large time-gap between corresponding Index values 18684 and 18686
# Both index and the date-time columns will be filled 

# Remove first row
sensor_date_time=sensor_date_time.iloc[1:]

start_time_str = '18-01-18 09:55'
end_time_str = '18-01-18 11:21'

start_time = pd.to_datetime(start_time_str)
end_time = pd.to_datetime(end_time_str)
time_difference_minutes = (end_time - start_time).seconds/ 60

temp_date_time=[]
temp_index=[]

# Probably a better way to loop and make a list
# new_date_times = [start_time + pd.Timedelta(minutes=minute) for minute in range(1, int(time_difference_minutes))]

for minute in range(1,int((time_difference_minutes))):
   # Data-Time column data filling
   temp_date_time.append(start_time + pd.Timedelta(minutes=minute))
   # Index column data filling
   temp_index.append("18685" + "-" + str(minute))

# Formatting the list to DataFrames
temp_date_time=pd.DataFrame({"date_time":temp_date_time})
temp_index=pd.DataFrame({"date_index":temp_index})
# Concatenate two temp df side by side, axis=1
concatenated_df = pd.concat([temp_index, temp_date_time], axis=1)

for files in uncleaned_sensor_groups_list:
   # Task 2:Load each sensor-group files in loops, overwrite 18685, and insert the temp_index, fill the rest of the columns with zero
   # Load uncleaned sensor files data in loop 

   # Laod the sensor file in loop
   temp_df=pd.read_csv(files)
   
   # Create a DataFrame with 85 rows and N columns of each sensor files and filled with zeros
   zero_df = pd.DataFrame(0, index=range(85), columns=range(len(temp_df.columns)-1))
   # Making the temp_date_time the size of the sensor group data 
   temp_index = pd.concat([temp_index, zero_df], axis=1)
   temp_index.columns=temp_df.columns

   # Concatenate the new df
   temp_df=pd.concat([temp_df[0:18684],temp_index,temp_df[18684:373037]], axis=0)
   
   # Task 3: Imputate the data frame

   # Replace 0 by NaN
   data_masked = temp_df.mask(temp_df == 0, np.nan)

   # Remove columns which has less than threshold amount of rows
   threshold_percentage = 80
   thresh_value = int((threshold_percentage / 100) * len(data_masked))
   # print(data_masked)
   temp_df = data_masked.dropna(axis=1, thresh=thresh_value)
   num_rows, num_columns = temp_df.shape
   print(f"0 filled {os.path.basename(files)} has #rows: {num_rows} #columns: {num_columns}")

   # Display the NaN percentage for each column and total
   nan_percentage = temp_df.isna().mean() * 100
   total_nan_percentage=nan_percentage.mean()
   # print(f"Each sensor NaN percentage:\n{nan_percentage} \nTotal NaN percentage: {total_nan_percentage}")

   # Impute NaN values using mice imputation

   print("Running Imputation...")

   #temp_df['index_date'] = pd.to_datetime(temp_df['index_date'])
   temp_df_without_index_date = temp_df.drop(columns=['index_date'])
 
   kds = mf.ImputationKernel(
      temp_df_without_index_date,
      datasets=1,
      save_all_iterations=True,
      random_state=1991)
   
    # Run the MICE algorithm for 3 iterations
   kds.mice(3)
 
   imputed_data_without_index_date = kds.complete_data()
   cleaned_data = pd.concat([temp_df['index_date'], imputed_data_without_index_date], axis=1)


   # Save the cleaned and imputated df as CSV
   cleaned_file_name=os.path.basename(files)
   cleaned_file_name=clean_sensor_data_dir + cleaned_file_name.split('.')[0] + " _CLN." + cleaned_file_name.split('.')[1]
   cleaned_data.to_csv(cleaned_file_name,index=False)

   # Set the temp index to one column by removing zero columns 
   temp_index=temp_index.iloc[:, 0:1]
   print("DataFrame saved as:", cleaned_file_name)
   


0 filled I-15 NB without Ramp.csv has #rows: 373122 #columns: 54
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/I-15 NB without Ramp _CLN.csv
0 filled I-15 SB without Ramp.csv has #rows: 373122 #columns: 55
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/I-15 SB without Ramp _CLN.csv
0 filled I-515 NB without Ramp.csv has #rows: 373122 #columns: 38
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/I-515 NB without Ramp _CLN.csv
0 filled I-515 SB without Ramp.csv has #rows: 373122 #columns: 37
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/I-515 SB without Ramp _CLN.csv
0 filled US-95 NB without Ramp.csv has #rows: 373122 #columns: 32
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/US-95 NB without Ramp _CLN.csv
0 filled US-95 SB without Ramp.csv has #rows: 373122 #columns: 34
Running Imputation...
DataFrame saved as: E:/I-15 Datataset/2018/cleaned/US-95 SB withou